In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sedmodels import Sedmodel
from mlmodels import train_encoder

In [4]:
model = Sedmodel(add_dust=False, add_neb=False, add_zred=True, add_burst=False)

Number of dimensions: 6

Filters:
  [<class 'sedpy.observate.Filter'>(sdss_u0), <class 'sedpy.observate.Filter'>(sdss_g0), <class 'sedpy.observate.Filter'>(sdss_r0), <class 'sedpy.observate.Filter'>(sdss_i0), <class 'sedpy.observate.Filter'>(sdss_z0)]

Free Parameters:
  ['zred', 'mass', 'logzsol', 'dust2', 'tage', 'tau']

Fixed Parameters:
  ['sfh', 'imf_type', 'dust_type']


# Compute grid of models

In [ ]:
if True: # Load model grid

    num_samples_per_dim = 6
    u_grid = np.load('data/u_grid_'+str(num_samples_per_dim)+'.npy')
    theta_grid = np.load('data/theta_grid_'+str(num_samples_per_dim)+'.npy')
    phot_grid = np.load('data/phot_grid_'+str(num_samples_per_dim)+'.npy')

    num_samples = u_grid.shape[0]
    print("\nNumber of samples:", num_samples)

else: # Create and write model grid

    if True:
        num_samples_per_dim = 6
        num_samples = num_samples_per_dim**model.num_dim
        grids = np.meshgrid(*[np.linspace(0, 1, num_samples_per_dim) for _ in range(model.num_dim)])
        u_grid = np.vstack([x.ravel() for x in grids]).T
        del grids
    else:
        num_samples = 10
        num_samples_per_dim = 0
        u_grid = np.random.uniform(size=num_samples*model.num_dim).reshape((num_samples, model.num_dim))
        
    print("\nNumber of samples:", num_samples)

    theta_grid, phot_grid, times = model.compute_photsed_grid(u_grid, plot_sed=True)
    
    np.save('data/u_grid_'+str(num_samples_per_dim)+'.npy', u_grid)
    np.save('data/theta_grid_'+str(num_samples_per_dim)+'.npy', theta_grid)
    np.save('data/phot_grid_'+str(num_samples_per_dim)+'.npy', phot_grid)
    
    print('Done in', np.sum(times), 'sec')
    
phot_grid = -2.5*np.log10(phot_grid)


Number of samples: 46656


In [ ]:
plt.hist(times)

## Prepare training and validation

In [ ]:
num_samples = u_grid.shape[0]
print("Number of samples:", num_samples)

#ind_train = np.arange(0, int(1*n_seds/8))
ind_all = np.arange(num_samples)
np.random.shuffle(ind_all)

split = int(0.8*num_samples)
ind_train = ind_all[:split]
ind_valid = ind_all[split:]

print("Number of training points:", ind_train.size)
print("Number of validation points:", ind_valid.size)

print("Dimension of parameter space", u_grid.shape[1])
print("Dimension of model space", phot_grid.shape[1])

train_params, train_models = u_grid[ind_train, :], phot_grid[ind_train, :]
valid_params, valid_models = theta_grid[ind_valid, :], phot_grid[ind_valid, :]

# Auto-encoder

In [ ]:
batch_size = 512
keep_prob = 0.8
num_epoch = 10000
num_latent = 5
learning_rate = 0.01
num_hiddens = [1000]

In [ ]:
plt.hist(train_models, log=True);

In [ ]:
train_latent, train_models_out, valid_latent, valid_models_out = \
    train_encoder(train_params, train_models, valid_params, valid_models,
                  num_latent, num_hiddens, num_epoch, learning_rate, batch_size, keep_prob)

## Verify reconstruction quality on training and validation (interpolation)

In [ ]:
plt.hist(train_models_out/train_models, log=True, range=[1e-2, 1e2])

In [ ]:
train_models_out/train_models

## Visualize projection of parameter space onto latent space